# Figure 3 (ii) - Learn c, A and b Jointly
In this notebook, we test our Deep Inverse Optimization Framework on the task of learning A, b and c jointly

This notebook contains three major parts: 
- I. initialize test instances based on baseline lps generated in *Figure 3_step1_BaselineLP Generation.ipynb*  
- II. Experiment (lean A, b and c jointly using deep_inv_opt package)
- III. Data Analysis (generating boxplot as Figure 3(ii) in our paper)

In [ ]:
# Copyright (C) to Yingcong Tan, Andrew Delong, Daria Terekhov. All Rights Reserved.

import numpy as np
import matplotlib.pyplot as plt
import deep_inv_opt as io
import deep_inv_opt.plot as iop
import scipy.optimize as opt
import random
import torch
from scipy.spatial import ConvexHull
import os
import time

# All generated files will be saved in the following directory
PATH = "C:/Users/Public/Downloads/"

def seed_random(seed = 0 ):
    np.random.seed(seed)
    random.seed(seed)
    
def MSE(x1, x2):
    x1 = io.tensor([x1]).view(-1,1)
    x2 = io.tensor([x2]).view(-1,1)
    err = torch.mean(torch.sum((x1 - x2)**2,0))
    
    return err.detach().numpy().ravel()

def ADG(c,x_predict, x_target):
    c_vector = io.tensor(c).view(-1,1)
    x_predict = io.tensor(x_predict).view(-1,1)
    x_target = io.tensor(x_target).view(-1,1)
    err = torch.abs(c_vector.t() @ (x_target - x_predict))
    
    return err.detach().numpy().ravel()

def feasibility(point, A_ub, b_ub, tolerance= 6):
    m,n = A_ub.shape
    feasibility_ub = np.round(A_ub @ point.reshape((n,1)) - b_ub.reshape((m,1)), tolerance)
    if (feasibility_ub <0).all():
        return "feasible"
    elif (feasibility_ub >0).any():
        return "infeasible" 

### Unility Functions
- **read_baselineLP**: read A_ub, b_ub and c from baseline LP
- **record_corrupt_LP**: record information of corrupt LP (the testing instance for task of learning A, b and c)
- **plot_LP**

In [ ]:
def read_baselineLP(filename):
    lines=[]
    
    with open(filename, "rt") as fp:
        for line in fp:
            lines.append(line.rstrip('\n')) 

    m,n = np.fromstring(lines[0], dtype=int, sep=' ')
    
    temp = [lines[i+1: i+1+m] for i in range(len(lines)) if "A_ub" in lines[i]]
    A_ub=np.zeros((m,n))
    b_ub=np.zeros((m,1))
    for j in range(m):
        A_ub[j] = np.fromstring(temp[0][j], dtype=float, sep=' ')
    
    temp = [lines[i+1: i+1+m] for i in range(len(lines)) if "b_ub" in lines[i]]
    for j in range(m):
        b_ub[j] = np.fromstring(temp[0][j], dtype=float, sep=' ')
        
    temp = [lines[i] for i in range(len(lines)) if "vertices" in lines[i]]
    num_vertices = int(temp[0].split()[0])

    temp = [lines[i+1: i+1+num_vertices] for i in range(len(lines)) if "vertices" in lines[i]]
    vertices = np.zeros((num_vertices,n))
    for j in range(num_vertices):
        vertices[j] = np.fromstring(temp[0][j], dtype=float, sep=' ')

    print(vertices)
    return A_ub, b_ub, vertices

def record_corrupt_LP(filename, data, target_type):
    (A_ub, b_ub, c_true, x_true, c_corrupt, x_corrupt) = data

    m,n = A_ub.shape
    with open(filename, 'w') as LP_file:
        # print LP size, m - # constrains, n - # # var        
        np.savetxt(LP_file, A_ub.shape, fmt="%.d")
        LP_file.write("\n")

        # print LP, Matrix A and b for constraints Ax <= b
        LP_file.write("A_ub\n")
        np.savetxt(LP_file, A_ub, fmt="%.6f")
        LP_file.write("b_ub\n")
        np.savetxt(LP_file, b_ub, fmt="%.6f")
        LP_file.write("\n")
        
        LP_file.write("c_true\n")
        np.savetxt(LP_file, c_true.reshape(1,nVar), fmt="%.6f")
        LP_file.write("x_true\n")
        np.savetxt(LP_file, x_true.reshape(1,nVar), fmt="%.6f")
        LP_file.write("\n")
        
        LP_file.write("c_corrupt\n")
        np.savetxt(LP_file, c_corrupt.reshape(1,nVar), fmt="%.6f")
        LP_file.write("x_corrupt %s\n"%target_type)
        np.savetxt(LP_file, x_corrupt.reshape(1,nVar), fmt="%.6f")
        
def plot_LP(filename, ind_lp, vertices, data):
    (A_ub, _, c_true, x_true, c_corrupt, x_corrupt) = data

    # plot feasible region in 2D
    m,n = A_ub.shape
    hull = ConvexHull(vertices) 
    fig = plt.figure(figsize=(5,5), dpi=100)
    ax = fig.add_subplot(111)
            
    for simplex in hull.simplices:
        plt.plot(vertices[simplex, 0], vertices[simplex, 1], 'k-')
    
    x0,y0 = x_true.ravel()
    cx,cy = 1.5*(c_true/sum(abs(c_true))).ravel()
    ax.arrow(x0,y0 ,cx,cy+0.1, color='b', head_width=0.2, length_includes_head=True)
    ax.text(cx + x0, cy +y0, "c_true", color='b',fontsize=10)                 
    ax.plot(x_true[0], x_true[1], 'bo', label='x_true')
                 
    x0,y0 = x_corrupt.ravel()
    cx,cy = 1.5*(c_corrupt/sum(abs(c_corrupt))).ravel()
    ax.arrow(x0,y0 ,cx,cy, color='r', head_width=0.2, length_includes_head=True)  
    ax.text(cx + x0, cy +y0+0.1, "c_corrupt", color='r',fontsize=10)
    ax.plot(x_corrupt[0], x_corrupt[1], 'ro', label='x_corrupt')  
    
    plt.axis("equal")
    plt.xlabel('X1', fontsize=10)
    plt.ylabel('X2', fontsize=10)
    plt.xlim(-5,5)
    plt.ylim(-5,5)
    plt.title("Corrupted LP %d (n=%d, m=%d)" % (ind_lp+1, n, m), fontsize =12)    
    plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1),
          fontsize = 12, ncol=2)
    plt.savefig(filename, framon = True,bbox_inches='tight')

## I. Generate Test Instances
- use the same convexhull as baseline LPs
- Generate a random c (which is recorded as c_true, then find its correponding solution (recorded as x_true)  
- "Corrupt" c_true by adding random noise  
- "Corrupt" x_true such that:  
 - it becomes infeasible (i.e., infeasible x_target):  
 add random noise to x_true, return x_target if it failed the feasiblity test (i.e., (A@x_target-b>0).any())
 - it becomes feasible (i.e., feasible x_target):  
 generate random convex combination of all vertices, and ensure the x_true has a relativly large coefficient. (i.e., x_fea = x1*w1+x2*w2+ ... +x_true*w_true, s.t. w1+w2+...+w_true = 1 and w_true = 0.9)

In [ ]:
def generate_3bLP(ind_lp, nVar, nCons, sgima, target_type):
    
    def randomize_C_Xtarget(A_ub, b_ub):
        m,n=A_ub.shape
        c = np.random.normal(0,1, n)
        c /= sum(abs(c))
        c_vector = io.tensor(c).view(-1,1)
        A_ub = io.tensor(A_ub)
        b_ub = io.tensor(b_ub).view(-1,1)
        sol  = io.linprog (c_vector, A_ub, b_ub, max_steps = 200, eps = 1e-7)
        if len(sol.size())>0:
            return c_vector, sol
        
    baselineLP = PATH + "Deep_Inv_Opt/LP baseline/n=%d,m=%d/%svar_LP%s.txt"%(nVar, nCons, str(nVar).zfill(2),str(ind_lp+1).zfill(2))
    A_ub, b_ub, vertices = read_baselineLP(baselineLP)
    c_vector, sol = randomize_C_Xtarget(A_ub, b_ub)

    if target_type =="infeasible":
        flag = False 
        while flag == False: 
            x_noise = io.tensor(np.random.uniform(sigma*-1, sigma,(nVar, 1)))
            x_corrupt = torch.add(sol, x_noise).numpy()

            feaCheck = feasibility(x_corrupt, A_ub, b_ub, tolerance= 5)
            if feaCheck == target_type:
                flag = True 
    elif target_type =="feasible":   
        dis = np.zeros(len(vertices))
        for i in range(len(vertices)):
            dis[i] = MSE(vertices[i], sol.detach().numpy())
        ind = np.argmin(dis)

        a = torch.distributions.Dirichlet(io.tensor([1. for i in range(len(vertices)-1)])).sample((1,)).numpy()
        a = np.divide(a,np.sum(a))/10
        a = np.insert(a, ind, 0.9)
        x_corrupt = (a @ vertices).reshape((nVar, 1))

    c_noise = io.tensor(np.random.uniform(sigma*-1, sigma,(nVar, 1)))
    c_corrupt = torch.add(c_vector, c_noise).numpy()
    
    directory = PATH + "Deep_Inv_Opt/figure3_ii/n=%d,m=%d/"%(nVar, nCons)
    if not os.path.exists(directory):
        os.makedirs(directory)
    LP3b = directory + "%svar_LP%s.txt"%(str(nVar).zfill(2), str(ind_lp+1).zfill(2))    
    data = (A_ub, b_ub, c_vector.detach().numpy(), sol.detach().numpy(),
            c_corrupt, x_corrupt)
    
    record_corrupt_LP(LP3b, data, target_type = target_type)
    if nVar <=2:
        plot_LP(LP3b[:-3], ind_lp, vertices, data)

    return target_type

""" uncomment the following lines to run a quick test of this cell  """

# print("==== Testing \"corrupt_lp\" ====")
# seed_random()
# ind_lp, nVar, nCons, sigma, target_type = 0, 2, 4, 0.2, "feasible"
# generate_3bLP(ind_lp, nVar, nCons, sigma, target_type)


### Run the following cell to generate the complete test instanes for Figure 3(ii)

#### Pick instance index randomly for two groups: lps with feasible targets and lps with infeasible targets
for each class, there are 50 instances (25 for feasible target, 25 for infeasible target)

In [ ]:
def sample_target_type(nVar_range, nCons_range):
    for i in range(len(nVar_range)):
        for j in range(len(nCons_range[i])):
            print("%d var, %d cons" %(nVar_range[i], nCons_range[i][j]))
            fea_temp = random.sample(range(num_lp), 25)
            infea_temp = [int(i)  for i in range(num_lp) if i not in fea_temp]
            fea_temp.sort()
            infea_temp.sort()
            print("\t",len(fea_temp),"feasible points\n\tindex", fea_temp)
            print("\t",len(infea_temp),"infeasible points\n\tindex", infea_temp)

            fea_target.append(fea_temp)
            infea_target.append(infea_temp)

    #         uncomment to record the index of fea and infea targets
    fea_file = PATH + "Deep_Inv_Opt/figure3_ii/index_fea_infea_targets.txt"
    with open(fea_file, "w") as file:
        ind = 0
        for i in range(len(nVar_range)):
            for j in range(len(nCons_range[i])):
                file.write("%d var, %d cons\n" %(nVar_range[i], nCons_range[i][j]))
                file.write("feasible target\n")
                np.savetxt(file, np.array(fea_target[ind]).reshape((1, len(fea_target[ind]))), fmt="%.d")
                file.write("infeasible target\n")
                np.savetxt(file, np.array(infea_target[ind]).reshape((1, len(infea_target[ind]))), fmt="%.d")
                file.write("\n")
                ind+=1

fea_target= []
infea_target = []  
nVar_range = [2, 10]
nCons_range = [[4, 8, 16],
               [20, 36, 80]]
num_lp = 50
print("==== \"sample_target_type\" ====")
sample_target_type(nVar_range, nCons_range)
      

### Run the following cell to generate the complete test instances for figure 3(ii) based on the instance index randomly selected above

In [ ]:
import re
nVar_range = [2, 10]
nCons_range = [[4, 8, 16],
               [20, 36, 80]]
num_lp = 50
sigma = .2

seed_random(0)

ind = 0
for i, nVar in enumerate (nVar_range):
    for nCons in nCons_range[i]:
        print("LPs with %d var and %d cons"%(nVar, nCons))
        for ind_lp in np.arange(num_lp):
            if ind_lp in fea_target[ind]:
                target_type = "feasible"
                print("\tind_lp%d, generate feasible target"%(ind_lp+1))
            elif ind_lp in infea_target[ind]:
                target_type = "infeasible"
                print("\tind_lp%d, generate infeasible target"%(ind_lp+1))
            assert target_type == "feasible" or target_type == "infeasible", "wrong target_type"
            generate_3bLP(ind_lp, nVar, nCons, sigma, target_type)
        
        ind+=1

## II. Experiment - Learn A, b and c jointly using Deep_Inv_Opt Package


### Unility Function
- **read_3bLP**: read instance data (e.g., x_target, c_inital, etc.)
- **read_targettype**: read the target type matrix, for example, fea_ind is a 6*25 matrix which contains the indices of all instances who has a feasible target point.

In [ ]:
def read_3bLP(filename):
    lines=[]
    
    with open(filename, "rt") as fp:
        for line in fp:
            lines.append(line.rstrip('\n')) 
    # read LP instance
    m = int (lines[0])  # num constraints (# rows)
    n = int (lines[1])  # num variables (# columns)
    
    temp = [lines[j+1:j+m+1] for j in range(len(lines)) if "A_ub" in lines[j]]
    A_ub=np.zeros((m,n))
    for i in range(m):
        A_ub[i] = np.fromstring(temp[0][i], dtype=float, sep=' ')
    
    temp = [lines[j+1:j+m+1] for j in range(len(lines)) if "b_ub" in lines[j]]
    b_ub=np.zeros((m,1))
    for i in range(m):
        b_ub[i] = np.fromstring(temp[0][i], dtype=float, sep=' ')
    
    temp = [lines[j+1] for j in range(len(lines)) if "c_true" in lines[j]]
    
    c_true= np.fromstring(temp[0], dtype=float, sep=' ')
    
    temp = [lines[j+1] for j in range(len(lines)) if "x_true" in lines[j]]
    x_true= np.fromstring(temp[0], dtype=float, sep=' ')
    
    temp = [lines[j+1] for j in range(len(lines)) if "c_corrupt" in lines[j]]
    c_corrupt= np.fromstring(temp[0], dtype=float, sep=' ')
    
    temp = [lines[j+1] for j in range(len(lines)) if "x_corrupt" in lines[j]]
    x_corrupt= np.fromstring(temp[0], dtype=float, sep=' ')

    return A_ub, b_ub, c_true, x_true, c_corrupt, x_corrupt


def read_targettype(filename):
    lines=[]
    
    with open(filename, "rt") as fp:
        for line in fp:
            lines.append(line.rstrip('\n')) 

    temp = [lines[j+1] for j in range(len(lines)) if "feasible target" in lines[j] and "infeasible target" not in lines[j]]
    fea_ind=np.zeros((2,3,25))
    temp_ind=0
    for i in range(2):
        for j in range(3):
            fea_ind[i,j,:] = np.fromstring(temp[temp_ind], dtype=float, sep=' ')
            temp_ind+=1
    
    temp = [lines[j+1] for j in range(len(lines)) if "infeasible target" in lines[j]]
    infea_ind=np.zeros((2,3,25))
    temp_ind=0
    for i in range(2):
        for j in range(3):
            infea_ind[i,j,:] = np.fromstring(temp[temp_ind], dtype=float, sep=' ')
            temp_ind +=1    
    
    return fea_ind, infea_ind

### sample Hyperparameters

In [ ]:
_RANGE_CONS = [[4,  8,  16],
               [20, 36, 80]]
_RANGE_VAR = [2, 10]
_RANGE_MU= [1.5, 5, 10, 20]
_RANGE_T0 = [0.5, 1, 5, 10]
_RANGE_LR_C = [0.1, 1, 10]
_RANGE_LR_Ab = [0.1, 1, 10]
_RANGE_EPS = [1e-5]
_LOSS_FUNCTION = ["ADG","MSE"]
_HYPERPARAM = (_RANGE_MU, _RANGE_T0, _RANGE_EPS, _RANGE_LR_C)


def sample_hyperparam(filename, num_set = 20, print_hyperparam = True):
    hyperParam = [[]]*num_set
    for i in range(20):
        mu = _RANGE_MU[np.random.randint(0, len(_RANGE_MU))]
        t0 = _RANGE_T0[np.random.randint(0, len(_RANGE_T0))]
        eps = _RANGE_EPS[np.random.randint(0, len(_RANGE_EPS))]
        lr_c = _RANGE_LR_C[np.random.randint(0, len(_RANGE_LR_C))]
        lr_Ab = _RANGE_LR_Ab[np.random.randint(0, len(_RANGE_LR_Ab))]
        hyperParam[i] = (mu, t0, eps, lr_c, lr_Ab)
        if print_hyperparam:
            print("hyperparam set %d"%(i))
            print(hyperParam[i])
        with open(filename, 'w') as file:
            for j in range(len(hyperParam)):
                file.write("hyperParam%s: "%(str(j+1).zfill(2)))
                np.savetxt(file, np.mat(hyperParam[j]).ravel(), fmt='%.6f')
            file.write("\n")
            

def read_hyperParam(filename):
    with open(filename, 'rt') as file:
        lines=file.readlines()
        temp = [lines[i] for i in range(len(lines)) if "hyperParam" in lines[i]]
#         hyperParam = np.zeros((20,4))
        hyperParam = [[]]*20
        for i in range(len(temp)):
            te = temp[i]
            te = te[14:]
            te = te[:-2]
            te = te.split(' ')
            hyperParam[i] = [float(j) for j in te]
    return hyperParam

print("==== Sample Hyperparameters values for Random Hyperparameter Search====")
seed_random()
filename = PATH + 'Deep_Inv_Opt/figure3_ii/Hyper_Param.txt'
sample_hyperparam(filename)
read_hyperParam(filename)

### Experiment
#### Please Note, finishing the entire experiments (i.e., 6 * 50 instances for Training with ADG and MSE) might take 1-2 days. You may see our paper for the experiment results directly.

In [ ]:
def exp_randomsearch(ind_lp, LPInstance, hyperParam, target_type):
    def record_result(filename, hyperpara, result):
        loss, time, c_final, x_final = result
        with open(filename, 'a') as file:
            mu, t0, eps, lr_c, lr_Ab = hyperParam[ind_search]
            temp = (mu, t0, eps, lr_c, lr_Ab, l, t)
            string = ("hyperparam"+str(ind_search+1).zfill(2)+" "+ str(temp) )
            file.write(str(string))
            file.write("\n")
            file.write("c_final ")
            np.savetxt(file, c.reshape(1,nVar), fmt='%.6f') 
            file.write("x_final ")
            np.savetxt(file, x.reshape(1,nVar), fmt='%.6f')    
        
    # initialize LP instance
    A_ub, b_ub, _, _, c_corrupt, x_corrupt = LPInstance
    
    nCons,nVar = A_ub.shape
    A_ub = io.tensor(A_ub)
    b_ub = io.tensor(b_ub).view(-1,1)  
    c_init = io.tensor(c_corrupt).view(-1,1)
    x_target = io.tensor(x_corrupt).view(-1,1)  

    print("||||| Ins %d (%d var, %d constraint)"
          %(ind_lp+1, nVar, nCons))
    print("||||| ",target_type, "target: ", x_target.detach().numpy().ravel())
    
    for loss in ["ADG","MSE"]:   
        
        # record LP characteristics
        _RESULT_FILE = PATH + 'Deep_Inv_Opt/figure3_ii/EXP_%svar_%scon_%s.txt'%(str(nVar).zfill(2),str(nCons).zfill(2), loss)
        with open(_RESULT_FILE, 'a') as file:
            file.write( "LP%s, %svar %scon:\n"
                       %(str(ind_lp+1).zfill(2),str(nVar).zfill(2),str(nCons).zfill(2)))
            file.write( "x_target %s\n"%target_type ) 
            np.savetxt(file, x_target.detach().numpy().reshape(1,nVar), fmt='%.6f')
            file.write("c_init: ")
            np.savetxt(file, c_init.numpy().reshape(1,nVar), fmt='%.6f')
                        
        # initialize loss function
        if loss =="ADG":
            loss_callback = io.abs_duality_gap
            eps_decay = False
        elif loss =="MSE":
            loss_callback = io.squared_error 
            eps_decay = True
        
        callback = None

        for ind_search in range(len(hyperParam)):
            # initialize hyper parameters
            mu, t0, eps, lr_c, lr_Ab = hyperParam[ind_search]
            print("loss func %s, HyperParam %d = (%.1f, %.1f, %.5f, %.1f, %.4f)"
                  %(loss, ind_search+1, mu, t0, eps, lr_c, lr_Ab))
            print("---- c_init ", c_init.numpy().ravel())
            print("---- EXP %d / %d: "%(ind_search+1, len(hyperParam)))
            start_time = time.time()
            
            solver = io.custom_linprog(t0 = t0, mu = mu, max_steps = 10000)

                
            try:
                c_final, _, _, err, x_predict, init_err = io.inverse_linprog(x_target, 
                                                            c_init,  A_ub, b_ub,
                                                            max_steps=200,
                                                            eps = eps, 
                                                            eps_decay = eps_decay,
                                                            learn_rate_c=lr_c,
                                                            learn_rate_ub = lr_Ab,
                                                            loss=loss_callback,
                                                            return_loss=True,
                                                            solver=solver)
                runtime = time.time() - start_time

                c = c_final.detach().numpy().ravel()
                x = x_predict.detach().numpy().ravel()
                l = err.detach().numpy().ravel()
                t = runtime

                print("---- Final loss", l)

                result = (l[0], t, c, x)
                record_result(_RESULT_FILE, hyperParam[ind_search], result)   


            except:# catch error and return failed hyperparam search
                print("find an error, return failed hyperparam search")
                c = np.ones((c_corrupt.shape))*999
                x = np.ones((c_corrupt.shape))*999
                l = 999
                t = 999                
                
                result = (l, t, c, x)
                record_result(_RESULT_FILE, hyperParam[ind_search], result)   
                           
                pass

        print()


_RANGE_CONS = [[4,  8,  16],
               [20, 36, 80]]
_RANGE_VAR = [2, 10]

NUM_INS = 50
fea_ind, infea_ind = read_targettype(PATH + "Deep_Inv_Opt/figure3_ii/index_fea_infea_targets.txt")

for i, nVar in enumerate(_RANGE_VAR):
    for j, nCons in enumerate(_RANGE_CONS[i]):
        for ind_lp in range(NUM_INS):
            filename = PATH + "Deep_Inv_Opt/figure3_ii/n=%d,m=%d/%svar_LP%s.txt"%(nVar, nCons, str(nVar).zfill(2),str(ind_lp+1).zfill(2))
            LPInstance = read_3bLP(filename)
            
            if ind_lp in fea_ind[i,j,:]:
                target_type = "feasible"
            elif ind_lp in infea_ind[i,j,:]:
                target_type = "infeasible"
            assert target_type == "feasible" or target_type == "infeasible", "wrong target_type"

            hyperParam = read_hyperParam(PATH + 'Deep_Inv_Opt/figure3_ii/Hyper_Param.txt')
            exp_randomsearch(ind_lp, LPInstance, hyperParam, target_type)


## III. Data Analysis

### Unility Function
- **read_finalresult**: read result from data file

In [ ]:
def read_finalresult(num_lp, nCons, nVar, loss):
    
    print("loss = ", loss)
    c_init = np.zeros((num_lp, nVar))

    # read c_init and LP ins
    for ind_lp in range(num_lp):
        filename = PATH + "Deep_Inv_Opt/figure3_ii/n=%d,m=%d/%svar_LP%s.txt"%(nVar, nCons, str(nVar).zfill(2),str(ind_lp+1).zfill(2))
#         A, b, opt_sol, c = LP_ins(filename)
        A,b,_,_,c_corrupt,x_corrupt = read_3bLP(filename) #new read function for 3b ins generted using idea 5

        c_init[ind_lp] = c_corrupt

    # read x_target
    x_target = np.zeros((num_lp, nVar))
    result = []
    c_final = []
    x_final = []

    with open(PATH + 'Deep_Inv_Opt/figure3_ii/FinalExp _%svar_%scon_%s.txt'
              %(str(nVar).zfill(2),str(nCons).zfill(2), loss), 'rt') as file:
        lines=file.readlines()
#         temp = [lines[i+1] for i in range(len(lines)) if "Vertex" in lines[i]]
    temp = [lines[i+1] for i in range(len(lines)) if "x_target" in lines[i]]

    for i in range(num_lp):
        te =temp[i]
        te = te.split(' ')
        te = [float(d) for d in te]
#             print(te)
        x_target[i] = te
#     print("x_target\n",x_target)

    #read exp results of every trail (mu, to, eps, lr_c, err, runtime)
    temp = [lines[i] for i in range(len(lines)) if "hyperparam" in lines[i]]
#         hyperParam = np.zeros((20,4))
    for i in range(len(temp)):
        te = temp[i]
        te = te[14:]
        te = te[:-2]

        te = te.split(',')
        te = [float(d) for d in te]
        result.append(te)
    result= np.mat(result)
#             print(result.shape)
    #from result find the index of the best trail for each LP ins
    best_ind = []
    for i in range(num_lp):
        best_ind.append(int ( i*20 + np.argmin(result[i*20:(i+1)*20,-2])))
    best_ind = np.array(best_ind)
#         print(best_ind)


    # read c_final for every trail
    temp = [lines[i] for i in range(len(lines)) if "c_final" in lines[i]]
#         hyperParam = np.zeros((20,4))
    for i in range(len(temp)):
        te = temp[i]
        te = te[8:]                
        te = te.split(' ')
        te = [float(d) for d in te]
        c_final.append(te)
    c_final= np.mat(c_final)
#         print(c_final)

    # read x_final for every trail
    temp = [lines[i] for i in range(len(lines)) if "x_final" in lines[i]]
#     print(temp)
    #         hyperParam = np.zeros((20,4))
    for i in range(len(temp)):
        te = temp[i]
        te = te[8:]                
        te = te.split(' ')
        te = [float(d) for d in te]
        x_final.append(te)
    x_final= np.mat(x_final)
#         print(x_final)
#             print(result[best_ind, -2])
    return result, c_init, c_final, best_ind, x_target, x_final

### Collect experiment results using the read_finalresult function
- collect final error (using ADG and MSE functions defined before)
- collect initial c and x to computer initial error (using ADG and MSE functions defined before)

In [ ]:
def Collect_Err(num_lp,  nVar, num_constraints, loss):
    err_final = np.zeros((num_lp,len(num_constraints)))
    err_init = np.zeros((num_lp, len(num_constraints)))
    fea_ind = [[]]*3
    infea_ind = [[]]*3
    for j,nCons in enumerate (num_constraints): 
        print("LPs with %dVar, %dCons"%(nVar,nCons))
        result, c_init, c_final, best_ind, x_target, x_final = read_finalresult (num_lp, nCons, nVar, loss)
#         print(x_target)
        err = result[best_ind,-2]
        err_final[:,j] = np.array(err).ravel()
#         print(err, err_final)
        
        fea_temp=[]
        infea_temp=[]
        for ind_lp in range(num_lp):
            filename = PATH + "Deep_Inv_Opt/figure3_ii/n=%d,m=%d/%svar_LP%s.txt"%(nVar, nCons, str(nVar).zfill(2),str(ind_lp+1).zfill(2))
            A,b,_,_,_,_ = read_3bLP(filename)
#             A, b, _, _ = LP_ins(filename)
            if feasibility(x_target[ind_lp], A, b) == "feasible":
#                 print("ins",ind_lp, "has fea target")
                fea_temp.append(ind_lp)
            if feasibility(x_target[ind_lp], A, b) == "infeasible":
                infea_temp.append(ind_lp)
            A_ub = io.tensor(A)
            b_ub = io.tensor(b).view(-1,1)
            # c_init is not normalized
            c_vector = io.tensor(c_init[ind_lp]).view(-1,1)
            c_vector/=sum(abs(c_vector))
            
            x_predict = io.linprog(c_vector, A_ub, b_ub, eps = 1e-6).detach().numpy()
            if loss == "ADG":            
                err_init[ind_lp, j] = ADG( c_vector.detach().numpy(), x_predict, x_target[ind_lp])
            elif loss == "MSE":            
                err_init[ind_lp, j] = MSE(x_predict, x_target[ind_lp])
            #             print("recompute init error", err_init[ind_lp], "final error",err[ind_lp])
        
        fea_ind[j] = fea_temp
        print("index of lp with fea target",fea_temp)
        infea_ind[j] = infea_temp
        print("index of lp with infea target",infea_temp)

        print("%d fea target, %d infea target"%(len(fea_temp),len(infea_temp)))
#     err_final = np.array(err_final).ravel()
#     err_init =np.array(err_init).ravel()
    return err_init, err_final, fea_ind,  infea_ind


num_lp = 50
var_range = [2,10]
cons_range = [[4, 8, 16],
             [20,36,80]]

ADG02_init, ADG02_final, fea02_ind,  infea02_ind = Collect_Err(num_lp, 2, cons_range[0], "ADG")

ADG10_init, ADG10_final, fea10_ind,  infea10_ind = Collect_Err(num_lp, 10, cons_range[1], "ADG")

MSE02_init, MSE02_final, fea02_ind,  infea02_ind = Collect_Err(num_lp, 2, cons_range[0], "MSE")

MSE10_init, MSE10_final, fea10_ind,  infea10_ind = Collect_Err(num_lp, 10, cons_range[1], "MSE")



### Create boxplot using the collected data from the previous cell

**Note, the final boxplots might not look identical to what we presented in the paper due to the randomization in baseline LP generation.**

In [ ]:
ADG_ERR = (ADG02_init, ADG02_final, ADG10_init, ADG10_final )
MSE_ERR = (MSE02_init, MSE02_final, MSE10_init, MSE10_final)

target_ind = (fea02_ind,  infea02_ind, fea10_ind,  infea10_ind )

def final_plot(err_data, target_ind):
    init02, final02, init10, final10 = err_data
    fea02_ind,  infea02_ind, fea10_ind,  infea10_ind = target_ind


    fea=[]
    for ind in (fea02_ind, fea10_ind):
        for i in range(len(ind)):
            fea.append(ind[i])
    infea=[]
    for ind in (infea02_ind, infea10_ind):
        for i in range(len(ind)):
            infea.append(ind[i])

    data02 = init02
    for i in range(3):
        data02 = np.insert(data02, i*2+1, final02[:,i], axis=1)
    data10 = init10
    for i in range(3):
        data10 = np.insert(data10, i*2+1, final10[:,i], axis=1)

    data_ = np.column_stack((data02,data10))
    data_ [data_<1e-10]=1e-10

    return fea, infea, data_


for _LOSS in ["MSE", "ADG"]:

    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(111)
    
    if _LOSS == "ADG":
        fea, infea, data = final_plot(ADG_ERR, target_ind)
        data [data<1e-8]=1e-8
        plt.ylim((1e-9,1e2))
        print(np.array(fea).shape)
    elif _LOSS == "MSE":
        fea, infea, data = final_plot(MSE_ERR, target_ind)
        data [data<1e-10]=1e-10
        plt.ylim((1e-11,1e2))
        print(np.array(fea).shape)

    _, nbox = data.shape    

    ind_init = np.arange(0,nbox,2)
    ind_final = np.arange(1,nbox,2)

    y_init = data[:, ind_init]
    x_init = np.random.uniform(1-.15, 1+0.15, size=len(data[:, ind_init]))
    y_final = data[:, ind_final]
    x_final = np.random.uniform(2-.15, 2+0.15, size=len(data[:, ind_final]))
    for i in np.arange(2, nbox,2):
        min = i+1-0.15
        max = i+1+0.15
        x_init = np.column_stack((x_init, np.random.uniform(i+1-.15, i+1+0.15, 
                                                            size=len(data[:, i]))))
    for i in np.arange(3, nbox,2):
        min = i+1-0.15
        max = i+1+0.15
        x_final = np.column_stack((x_final, np.random.uniform(i+1-.15, i+1+0.15, 
                                                              size=len(data[:, i]))))
    ax.boxplot(data,sym='',medianprops =dict(linewidth=1,color='black'))

    for i in range(nbox//2):
        ax.plot(x_init[fea[i], i], y_init[fea[i], i], 
                ls='None',mec = 'g', marker="+", mew= 1, markersize=11, alpha=0.5) 
        ax.plot(x_init[infea[i], i], y_init[infea[i], i], 
                ls='None',mec = 'g', marker="x", mew= 1, markersize=9,alpha=0.5) 

    for i in range(nbox//2):
        ax.plot(x_final[fea[i], i], y_final[fea[i], i], 
               ls='None',mec = 'b', marker="+", mew= 1, markersize=11, alpha=0.5) 
        ax.plot(x_final[infea[i], i], y_final[infea[i], i], 
               ls='None',mec = 'b', marker="x", mew= 1, markersize=9, alpha=0.5) 
    
    plt.xticks([], [])
    plt.yscale("log")
    plt.yticks(fontsize=12)

    plt.savefig(PATH + "Deep_Inv_Opt/deepinverse_fig3_ii_%s.pdf"%_LOSS, frameon = True, bbox_inches='tight', dpi=100)
    plt.show()
    